# Production Code

In [ ]:
import os
import re
import numpy as np
import pandas as pd

source_directory = "../surveys/plain_txt/"

## Extract

### Global Variables

In [ ]:
column_headers = ["order_number", "renumerations_number_of_month",
                  "renumerations_monthly_amount", "transfers_number_of_times",
                  "transfers_unit_amount", "subsidies_number_of_month",
                  "subsidies_monthly_amount", "other"]

### Helper Functions

In [ ]:
def find_between(str_1, str_2, contents):
    regex_between = str_1 + r"([\w\W]*)" + str_2
    regex_search = re.findall(regex_between, contents)

    return regex_search

### Parsing Functions

In [ ]:
def parse_interviewee(contents):
    name_regex = r"nombre de la persona[^\n:]*:[\s]*([^\n]*)\n"
    name_search = re.findall(name_regex, contents)
    if len(name_search) == 1:
        name = name_search[0]
        if name == "3":
            name = "N/A"
    else:
        alt_name_regex = r"nombre de la persona[^\n\(]*\(\w* +([^\n]*)\n"
        alt_name_search = re.findall(alt_name_regex, contents)
        if len(alt_name_search) == 1:
            name = alt_name_search[0]
            if name == "3":
                name = "N/A"
        else:
            name = "UNENCOUNTERED FORMAT"

    name = name.strip()

    return name


def parse_date_range(section_search):
    date_range = ""

    if len(section_search) == 1:
        section_string = section_search[0]

        date_regex = r"([\w ]+\d+ *[-–] *[\w ]+\d+)"
        date_search = re.findall(date_regex, section_string)

        if len(date_search) == 1:
            date_range = date_search[0]
        else:
            date_range = "UNENCOUNTERED FORMAT"
    else:
        date_range = "UNENCOUNTERED FORMAT"

    return date_range


def parse_section_5_4(section_search, column_headers):
    section_5_4_data = []

    if len(section_search) == 1:
        section_string = section_search[0] + "sección vi:"

        table_start = r"monto mensual\n{4}"
        table_end = r"sección vi:"
        table_search = find_between(table_start, table_end, section_string)

        table_otros_start = r"monto mensual\n{2}\d+\n{2}"
        table_otros_search = find_between(table_otros_start, table_end,
                                          section_string)

        entry = {}

        if len(table_search) == 1:
            table_string = table_search[0]

            if re.search("\n", table_string):
                table_list = table_string.split("\n")

                # count until 16, skip odd indexes (1, 3, 5, 7, ...)
                count = 0
                for item in table_list:
                    item = item.strip()
                    if count == 16:
                        section_5_4_data.append(entry)
                        entry = {}
                        count = 0
                    if count % 2 == 0:
                        col = column_headers[count//2]
                        entry[col] = item
                    count += 1
            else:
                for col in column_headers:
                    entry[col] = "UNENCOUNTERED FORMAT"
                section_5_4_data.append(entry)
        elif len(table_otros_search) == 1:

            table_string = table_otros_search[0]

            if re.search("\n", table_string):
                table_list = table_string.split("\n")

                # count until 16, skip odd indexes (1, 3, 5, 7, ...)
                count = 0
                for item in table_list:
                    item = item.strip()
                    if count == 16:
                        section_5_4_data.append(entry)
                        entry = {}
                        count = 0
                    if count % 2 == 0:
                        col = column_headers[count//2]
                        entry[col] = item
                    count += 1
            else:
                for col in column_headers:
                    entry[col] = "UNENCOUNTERED FORMAT"
                section_5_4_data.append(entry)

            # capture otros amount
            entry = {}
            otros_search = re.findall(r"monto mensual\n{2}(\d+)\n{2}",
                                      section_string)

            if len(otros_search) == 1:
                for col in column_headers:
                    if col == "other":
                        entry[col] = otros_search[0]
                    else:
                        entry[col] = ""
                section_5_4_data.append(entry)
        else:
            for col in column_headers:
                entry[col] = "UNENCOUNTERED FORMAT"
            section_5_4_data.append(entry)
    else:
        section_5_4_data = [{}]

    return section_5_4_data

### Process the Whole File

In [ ]:
def process_files(source_directory, column_headers):
    dataset = []

    for process_file in os.listdir(source_directory):
        file_path = os.path.join(source_directory, process_file)

        # with statements automatically control the closing of files
        with open(file_path, "r") as file:
            contents = file.read()
            contents = contents.lower()

            section_start = "v-4: otros ingresos"
            section_end = "sección vi:"
            section_search = find_between(section_start, section_end, contents)

            interviewee = parse_interviewee(contents)
            date_range = parse_date_range(section_search)
            section_5_4_data = parse_section_5_4(section_search, column_headers)

            data_dict = {}

            # check to make sure files contain the right amount of entries
            if len(section_5_4_data) <= 1:
                print(f"Unencountered Format: {process_file}\n"
                      + f"Length of section_data: {len(section_5_4_data)}\n")

            for row in section_5_4_data:
                data_dict = {}
                data_dict["filename"] = process_file
                data_dict["interviewee"] = interviewee
                data_dict["date_range"] = date_range

                for key, value in row.items():
                    data_dict[key] = value

                dataset.append(data_dict)

    # convert list to DataFrame
    raw_df = pd.DataFrame(data=dataset)

    return raw_df

## Transform

### Helper Functions

In [ ]:
def clean_date_range(dataf):
    replace_dict = {"agosto 2018 - agosto 2019": "august 2018-2019",
                    "septiembre 2018 - septiembre 2019": "september 2018-2019",
                    "octubre 2018 – octubre 2019": "october 2018-2019",
                    "octubre 2018 - octubre 2019": "october 2018-2019",
                    "octubre 2018 - octubre2019": "october 2018-2019",
                    "octubre 2018 – octubre  2019": "october 2018-2019"}
    dataf["date_range"] = dataf["date_range"].replace(replace_dict)

    return dataf


def clean_order_number(dataf):
    replace_dict = {"01": "1",
                    "02": "2"}
    dataf["order_number"] = dataf["order_number"].replace(replace_dict)

    return dataf


def clean_amounts(dataf):
    amount_cols = ["renumerations_monthly_amount",
                   "transfers_unit_amount",
                   "subsidies_monthly_amount"]

    for col in amount_cols:
        # handle thousands separator
        replace_filter = dataf[col].str.contains(r"\.\d{3}", na=False)
        replace_series = dataf[col].str.replace(".", "")

        dataf[col] = dataf[col].where(~replace_filter, replace_series)

        # drop $ from amount
        dataf[col] = dataf[col].str.replace("$", "")
        dataf[col] = dataf[col].str.replace("*", "")

    return dataf


def clean_runumeration_num_months(dataf):
    replace_dict = {"4 al 11": "4-11",
                    "11 al 4": "11-4",
                    "4 al 10": "4-10",
                    "3 al 11": "3-11",
                    "4 al 12": "4-12",
                    "6 al 12": "6-12",
                    "3 al 10": "3-10",
                    "2 al 10": "2-10",
                    "07 al 12": "7-12",
                    "4 al 7": "4-7",
                    "12 meses": "12",
                    "5 al 10": "5-10",
                    "6 meses": "6",
                    "4 al 8": "4-8"}
    dataf["renumerations_number_of_month"] = dataf["renumerations_number_of_month"].replace(replace_dict)

    return dataf


def clean_transfer_num_times(dataf):
    replace_dict = {"ninguno": "0",
                    "2 veces x año": "2",
                    "ningun": "0",
                    "2 x año": "2"}
    dataf["transfers_number_of_times"] = dataf["transfers_number_of_times"].replace(replace_dict)

    return dataf


def clean_subsidies_num_month(dataf):
    replace_dict = {"ninguno": "0"}
    dataf["subsidies_number_of_month"] = dataf["subsidies_number_of_month"].replace(replace_dict)

    return dataf

### Pipeline Functions

In [ ]:
def start_pipeline(dataf):
    return dataf.copy()


def handle_null_data(dataf):
    # fill missing values with null
    dataf = dataf.replace("-", np.NaN)
    dataf = dataf.replace("", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = isnull_sum >= 8
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf


def clean_values(dataf):
    dataf = clean_date_range(dataf)
    dataf = clean_order_number(dataf)
    dataf = clean_amounts(dataf)
    dataf = clean_runumeration_num_months(dataf)
    dataf = clean_transfer_num_times(dataf)
    dataf = clean_subsidies_num_month(dataf)

    return dataf

### Create Raw and Cleaned DataFrame

In [ ]:
raw_df = process_files(source_directory, column_headers)

clean_df = (raw_df
            .pipe(start_pipeline)
            .pipe(clean_values)
            .pipe(handle_null_data))

## Other
### Functions to Check the Implementation

In [ ]:
def get_dropped_df(raw_dataf, clean_dataf):
    raw_indexes = raw_dataf.index
    clean_indexes = clean_dataf.index

    dropped_indexes = raw_indexes[~raw_indexes.isin(clean_indexes)]
    dropped_df = raw_dataf.loc[dropped_indexes].copy()

    return dropped_df

### Create Dropped DataFrame

In [ ]:
dropped_df = get_dropped_df(raw_df, clean_df)

## Load
### Export to CSV

In [ ]:
datasets_directory = "../datasets/"
filename = "section_5_4.csv"
file_path = os.path.join(datasets_directory, filename)

clean_df.to_csv(file_path, index=False, na_rep="null")

***
# Testing Code
### View the Data

In [ ]:
clean_df.head(50)

### Check for Parsing Errors

In [ ]:
cols_to_check = clean_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(clean_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Check Dropped Rows
Another way to check is by opening the variable inspector and manually scrolling
through the `df_dropped` DataFrame. 

The code below tells us whether or not there are files that contain no information for this section. If raw file count is the same as clean file count, then no files were dropped.

In [ ]:
print("Raw File Count: {}".format(len(raw_df["filename"].value_counts())))
print("Clean File Count: {}".format(len(clean_df["filename"].value_counts())))

If there are files that are completely dropped, find those files and look at them to check for parsing errors.

In [ ]:
raw_file_names = raw_df["filename"].value_counts().index
clean_file_names = clean_df["filename"].value_counts().index

for file in raw_file_names[~raw_file_names.isin(clean_file_names)]:
    print(file)

Check the rows value counts to see if any rows are being dropped that should not be.

In [ ]:
cols_to_check = dropped_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(dropped_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Duplicate Rows

In [ ]:
df_columns = clean_df.columns[clean_df.columns != "filename"]
clean_df[clean_df.loc[:, df_columns].duplicated(keep=False)]